In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm
%matplotlib inline
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.feature_selection import RFE
from sklearn.linear_model import RidgeCV, LassoCV, Ridge, Lasso

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import sklearn

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
df = pd.read_csv("/kaggle/input/dacc-jk/SG_Part0.csv")
df

In [ ]:
for i in range(1,10):
    df1 = pd.read_csv("/kaggle/input/dacc-jk/SG_Part"+str(i)+".csv")
    df = pd.concat([df,df1])
df

In [ ]:
dff = df.sample(n=1000000,random_state = 1)
dff_tt = df.sample(n = 10000000, random_state = 9283)
dff

In [ ]:
df["osname"].value_counts()

In [ ]:
dff["osname"] = dff["osname"].replace(["ios","android"],[1,-1])
dff["driving_mode"] = dff["driving_mode"].replace(["car"],[1])
dff["osname"]

In [ ]:
dff_tt["osname"] = dff_tt["osname"].replace(["ios","android"],[1,-1])
dff_tt["driving_mode"] = dff_tt["driving_mode"].replace(["car"],[1])
dff_tt["osname"]

In [ ]:
dff = dff.drop(columns=["trj_id","pingtimestamp"])
dff_y = dff["accuracy"]
dff = dff.drop(columns=["accuracy"])
dff_y

In [ ]:
dff_tt = dff_tt.drop(columns=["trj_id","pingtimestamp"])
dff_ytt = dff_tt["accuracy"]
dff_tt = dff_tt.drop(columns=["accuracy"])
dff_ytt

In [ ]:
dff.columns

Using Linear Regression

In [ ]:
model = LinearRegression()
model.fit(dff,dff_y)
importance = model.coef_
for i,v in zip(dff.columns,importance):
	print('Feature: %s, Score: %.5f' % (i,v))
# plot feature importance
plt.bar([x for x in dff.columns], importance)
plt.show()
plt.savefig("linear_reg.png")

In [ ]:
from sklearn.metrics import mean_squared_error

In [ ]:
dff_tt_yy = model.predict(dff_tt)
mean_squared_error(dff_ytt,dff_tt_yy)

In [ ]:
plt.scatter(dff_ytt, dff_tt_yy)
plt.title('True vs Predicted')
plt.xlabel('True')
plt.ylabel('Predicted')
plt.show()

Using K Neightbours Regressor to get feature importance

In [ ]:
from sklearn.neighbors import KNeighborsRegressor
from sklearn.inspection import permutation_importance
model = KNeighborsRegressor(weights="distance",leaf_size = 3,p = 5, )
# fit the model
model.fit(dff,dff_y)
# perform permutation importance
results = permutation_importance(model, dff,dff_y, scoring='neg_mean_squared_error')
# get importance
importance = results.importances_mean
# summarize feature importance
for i,v in zip(dff.columns,importance):
	print('Feature: %s, Score: %.5f' % (i,v))
# plot feature importance
plt.bar([x for x in dff.columns], importance)
plt.show()

In [ ]:
dff_tt_yy = model.predict(dff_tt)
mean_squared_error(dff_ytt,dff_tt_yy)

In [ ]:
results.importances

Decision Tree Regressor

In [ ]:
from sklearn.model_selection import cross_val_score
from sklearn.tree import DecisionTreeRegressor
modelTree = DecisionTreeRegressor(max_depth = 2)
modelTree.fit(dff,dff_y)
results = permutation_importance(modelTree, dff,dff_y, scoring='neg_mean_squared_error')
# get importance
importance = results.importances_mean
# summarize feature importance
for i,v in zip(dff.columns,importance):
	print('Feature: %s, Score: %.5f' % (i,v))
# plot feature importance
plt.bar([x for x in dff.columns], importance)
plt.show()

In [ ]:
dff_tt_yy = modelTree.predict(dff_tt)
mean_squared_error(dff_ytt,dff_tt_yy)

Various graphs to plot relationship